In [ ]:
!pip install playwright pandas openpyxl tqdm
!playwright install
!playwright install --with-deps chromium
!pip install nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 13.5 MB/s eta 0:00:00
167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 64.2s167.7 MiB [] 0% 34.8s167.7 MiB [] 0% 23.5s167.7 MiB [] 0% 14.0s167.7 MiB [] 0% 10.2s167.7 MiB [] 1% 8.2s167.7 MiB [] 1% 7.1s167.7 MiB [] 2% 6.4s167.7 MiB [] 2% 6.2s167.7 MiB [] 2% 5.6s167.7 MiB [] 3% 5.3s167.7 MiB [] 3% 5.0s167.7 MiB [] 4% 4.7s167.7 MiB [] 4% 4.5s167.7 MiB [] 5% 4.4s167.7 MiB [] 5% 4.3s167.7 MiB [] 5% 4.7s167.7 MiB [] 6% 4.6s167.7 MiB [] 6% 4.7s167.7 MiB [] 7% 4.8s167.7 MiB [] 7% 4.7s167.7 MiB [] 8% 4.5s167.7 MiB [] 8% 4.4s167.7 MiB [] 9% 4.4s167.7 MiB [] 10% 4.3s167.7 MiB [] 11% 4.0s167.7 MiB [] 12% 3.9s167.7 MiB [] 12% 4.0s167.7 MiB [] 13% 3.9s167.7 MiB [] 14% 3.7s167.7 MiB [] 14% 3.6s167.7 MiB [] 15% 3.6s167.7 MiB [] 16% 3.5s167.7 MiB [] 17% 3.4s167.7 MiB [] 18% 3.3s167.7 MiB [] 18% 3.2s167.7 MiB [] 19% 3.1s167.7 MiB [] 20% 3.1s167.7 MiB [] 21% 3.0s167.7 MiB [] 22% 2.9s167.7 MiB [] 23% 2.9s167.7 MiB [] 24% 2.8s167.7 MiB [] 25% 2.7s167.7 MiB [] 26% 

In [ ]:
# import thư viện
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from tqdm.asyncio import tqdm_asyncio # Để hiển thị thanh tiến trình
import re # Import thư viện regex để làm sạch text

# Cấu hình
BASE_URL = "https://careerviet.vn/viec-lam/tat-ca-viec-lam-vi.html"
OUTPUT_FILE = "careerviet_jobs.xlsx"
JOB_ITEM_SELECTOR = "div.job-item"
JOB_TITLE_SELECTOR = "a.job_link"
COMPANY_NAME_SELECTOR = "div.caption a.company-name"
LOCATIONS_SELECTOR = "div.location ul li"
SALARY_SELECTOR = "div.salary p"
DEADLINE_DATE_SELECTOR = "div.time ul li:has(em.fa-clock-o) time" # Hạn nộp
UPDATE_DATE_SELECTOR = "div.time ul li:has(em.mdi-calendar) time" # Ngày cập nhật



In [ ]:
async def get_element_text(element, selector, default="N/A"):
    if not element: return default
    try:
        selected_element = await element.query_selector(selector)
        if selected_element:
            return await selected_element.inner_text()
        return default
    except Exception as e:
        print(f"Lỗi nhỏ khi lấy text với selector '{selector}': {e}")
        return default



In [ ]:
async def get_element_attribute(element, selector, attribute, default="N/A"):
    if not element: return default
    try:
        selected_element = await element.query_selector(selector)
        if selected_element:
            return await selected_element.get_attribute(attribute)
        return default
    except Exception as e:
        print(f"Lỗi nhỏ khi lấy attribute '{attribute}' với selector '{selector}': {e}")
        return default



In [ ]:
async def scrape_page(page, page_num):
    if page_num == 1:
        url = BASE_URL
    else:
        url = f"{BASE_URL}/trang-{page_num}"

    print(f"Đang cào dữ liệu từ trang: {url}")
    try:
        await page.goto(url, wait_until="networkidle", timeout=90000)
        await page.wait_for_timeout(5000)

        print(f"Đang chờ selector chính cho các mục công việc: '{JOB_ITEM_SELECTOR}'...")
        await page.wait_for_selector(JOB_ITEM_SELECTOR, timeout=70000)
        print(f"Đã tìm thấy selector chính: '{JOB_ITEM_SELECTOR}'.")

    except Exception as e:
        print(f"Lỗi khi tải hoặc tìm selector '{JOB_ITEM_SELECTOR}' trên trang {url}: {e}")
        return []

    job_items = await page.query_selector_all(JOB_ITEM_SELECTOR)
    page_data = []

    if not job_items:
        print(f"Không tìm thấy công việc nào với selector '{JOB_ITEM_SELECTOR}' trên trang: {url}.")
        return []

    print(f"Tìm thấy {len(job_items)} mục công việc trên trang {page_num}.")

    for item_index, item in enumerate(job_items):
        job_title = (await get_element_text(item, JOB_TITLE_SELECTOR) or "").strip()
        job_url_raw = await get_element_attribute(item, JOB_TITLE_SELECTOR, "href")
        job_url = "https://careerviet.vn" + job_url_raw if job_url_raw and not job_url_raw.startswith("http") else job_url_raw

        # Lấy tên công ty
        company_name_raw = await get_element_text(item, COMPANY_NAME_SELECTOR)
        company_name = (company_name_raw or "").strip()



        # Lấy địa điểm
        locations_elements = await item.query_selector_all(LOCATIONS_SELECTOR)
        locations = [ (await loc.inner_text() or "").strip() for loc in locations_elements]
        location_str = ", ".join(filter(None, locations)) if locations else "N/A"

        # Lấy mức lương và làm sạch
        salary_raw = await get_element_text(item, SALARY_SELECTOR)
        salary = (salary_raw or "").strip()
        if "Lương :" in salary: # Loại bỏ tiền tố "Lương :"
            salary = salary.replace("Lương :", "").strip()
        if not salary: salary = "N/A"


        # Lấy ngày đăng/hạn nộp
        posted_date_raw = await get_element_text(item, DEADLINE_DATE_SELECTOR) # Ưu tiên hạn nộp
        if not posted_date_raw or posted_date_raw == "N/A":
            posted_date_raw = await get_element_text(item, UPDATE_DATE_SELECTOR) # Nếu không có hạn nộp, lấy ngày cập nhật

        posted_date = (posted_date_raw or "").strip()
        if not posted_date: posted_date = "N/A"


        page_data.append({
            "TieuDeCongViec": job_title if job_title else "N/A",
            "TenCongTy": company_name if company_name else "N/A",
            "DiaDiem": location_str if location_str else "N/A",
            "MucLuong": salary, # Đã xử lý "N/A"
            "HanNop": posted_date, # Đã xử lý "N/A"
            "URL": job_url if job_url else "N/A",
        })
    return page_data



In [ ]:
async def main_scraper(num_pages_to_scrape: int):
    all_jobs_data = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
            java_script_enabled=True,
            accept_downloads=False,
        )
        page = await context.new_page()

        print(f"Sẽ cào {num_pages_to_scrape} trang.")
        for i in tqdm_asyncio(range(num_pages_to_scrape)):
            page_num = i + 1
            data = await scrape_page(page, page_num)
            if data:
                all_jobs_data.extend(data)
            if i < num_pages_to_scrape - 1:
                 print(f"Nghỉ 5 giây trước khi cào trang tiếp theo...")
                 await asyncio.sleep(5)

        await browser.close()

    df = pd.DataFrame(all_jobs_data)

    if df.empty:
        print("\nKhông có dữ liệu nào được cào. Vui lòng kiểm tra lại các SELECTOR và các thông báo lỗi.")
    else:
        try:
            df.to_excel(OUTPUT_FILE, index=False, engine='openpyxl')
            print(f"\nĐã cào xong {len(all_jobs_data)} công việc.")
            print(f"Dữ liệu đã được lưu vào file: {OUTPUT_FILE}")
        except Exception as e:
            print(f"Lỗi khi lưu file Excel: {e}")
            print("Đang lưu sang CSV...")
            try:
                csv_output_file = OUTPUT_FILE.replace(".xlsx", ".csv")
                df.to_csv(csv_output_file, index=False, encoding='utf-8-sig')
                print(f"Dữ liệu đã được lưu vào file: {csv_output_file}")
            except Exception as e_csv:
                print(f"Lỗi khi lưu file CSV: {e_csv}")
    return df



SO_TRANG_CAN_CAO = 1
scraped_data_df = await main_scraper(SO_TRANG_CAN_CAO)

if not scraped_data_df.empty:
    print("\nXem trước dữ liệu (5 dòng đầu):")
    display(scraped_data_df.head())
else:
    print("\nDataFrame rỗng, không có dữ liệu để hiển thị.")

Sẽ cào 1 trang.


  0%|          | 0/1 [00:00<?, ?it/s]

Đang cào dữ liệu từ trang: https://careerviet.vn/viec-lam/tat-ca-viec-lam-vi.html
Đang chờ selector chính cho các mục công việc: 'div.job-item'...
Đã tìm thấy selector chính: 'div.job-item'.
Tìm thấy 50 mục công việc trên trang 1.


100%|██████████| 1/1 [00:44<00:00, 44.41s/it]


Đã cào xong 50 công việc.
Dữ liệu đã được lưu vào file: careerviet_jobs.xlsx

Xem trước dữ liệu (5 dòng đầu):


,TieuDeCongViec,TenCongTy,DiaDiem,MucLuong,HanNop,URL
0,TRƯỞNG PHÒNG MARKETING (MỚI),CÔNG TY TNHH BBQ VIỆT NAM,Hà Nội,Lương: 30 Tr - 35 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/truong-p...
1,CVCC Xử Lý Tín Dụng Khách Hàng Doanh Nghiệp Lớ...,Ngân Hàng TMCP Việt Nam Thịnh Vượng - VPBANK,Hồ Chí Minh,Lương: Cạnh tranh,10-07-2025,https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-...
2,Nhân Viên Vận Hành Kho (MỚI),CÔNG TY CỔ PHẦN KẾT CẤU THÉP REMARK,Đồng Nai,Lương: Cạnh tranh,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...
3,Cộng tác viên Kinh doanh (MỚI),CÔNG TY CỔ PHẦN DI CHUYỂN VÀ THÔNG MINH GSM,Hồ Chí Minh,Lương: 8 Tr - 12 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cong-tac...
4,Operation Manager (MỚI),CÔNG TY TNHH DỊCH VỤ THƯƠNG MẠI CUNG ỨNG AG,Hồ Chí Minh,Lương: 13 Tr - 15 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/operatio...


In [ ]:

from playwright.async_api import Page # Cần thiết cho type hinting

async def extract_generic_section(page: Page, section_title_text: str) -> str:
    """
    Trích xuất nội dung từ một mục cụ thể trên trang (ví dụ: Mô tả, Yêu cầu)
    dựa vào tiêu đề của mục đó.
    Hàm này giả định trang đã được điều hướng đến URL chính xác.
    """
    print(f"INFO: Đang tìm và trích xuất mục: '{section_title_text}'")

    # 1. Tìm thẻ h2 chứa tiêu đề mục cần trích xuất
    h2_title_elements = await page.query_selector_all("h2.detail-title")
    h2_section = None
    normalized_target_title = section_title_text.lower().strip()

    for h2_candidate in h2_title_elements:
        text_content = await h2_candidate.inner_text()
        if normalized_target_title in text_content.lower().strip():
            h2_section = h2_candidate
            break

    if not h2_section:
        print(f"LỖI: Không tìm thấy tiêu đề cho mục '{section_title_text}'.")
        return "N/A"

    # 2. Từ thẻ h2, tìm div cha chứa nội dung của mục
    section_container = await h2_section.query_selector("xpath=ancestor::div[contains(@class, 'detail-row') and contains(@class, 'reset-bullet')]")

    if not section_container:
        print(f"LỖI: Không tìm thấy khối (container) chứa nội dung cho mục '{section_title_text}'.")
        return "N/A"

    # 3. Lấy nội dung
    content_parts = []

    # Ưu tiên 1: Tìm các thẻ <p>
    p_elements = await section_container.query_selector_all("p")
    if p_elements:
        for p_el in p_elements:
            text = await p_el.inner_text()
            cleaned_text = text.strip()
            if cleaned_text:
                content_parts.append(cleaned_text)

    if content_parts:
        print(f"INFO: Đã lấy nội dung cho '{section_title_text}' từ thẻ <p>.")
    else:
        # Ưu tiên 2: Nếu không có <p>, tìm thẻ <li>
        print(f"INFO: Không tìm thấy nội dung trong thẻ <p> cho '{section_title_text}'. Thử tìm thẻ <li>.")
        li_elements = await section_container.query_selector_all("li")
        if li_elements:
            for li_el in li_elements:
                text = await li_el.inner_text()
                cleaned_text = text.strip()
                if cleaned_text:
                    content_parts.append(cleaned_text)

            if content_parts:
                 print(f"INFO: Đã lấy nội dung cho '{section_title_text}' từ thẻ <li>.")

    # Ưu tiên 3: Fallback - inner_text của container (cố gắng trừ H2)
    if not content_parts:
        print(f"CẢNH BÁO: Không có nội dung từ <p> hoặc <li> cho '{section_title_text}'. Sử dụng inner_text của container làm giải pháp cuối.")

        all_text_in_container = await section_container.inner_text()
        h2_text_content_for_removal = await h2_section.inner_text() # Text của H2 đã tìm thấy

        processed_text = all_text_in_container.strip()
        h2_stripped_text_for_removal = h2_text_content_for_removal.strip()

        # Cố gắng loại bỏ text của H2 nếu nó nằm ở đầu (không phân biệt hoa thường)
        if processed_text.lower().startswith(h2_stripped_text_for_removal.lower()):
            processed_text = processed_text[len(h2_stripped_text_for_removal):].strip()

        if processed_text:
            fallback_lines = [line.strip() for line in processed_text.splitlines() if line.strip()]
            if fallback_lines:
                content_parts.extend(fallback_lines)
                print(f"INFO: Đã sử dụng giải pháp cuối (inner_text của container, đã cố loại bỏ H2) cho '{section_title_text}'.")
            elif all_text_in_container.strip():
                 content_parts.append(all_text_in_container.strip()) # Lấy text gốc
                 print(f"INFO: Đã sử dụng giải pháp cuối (inner_text gốc của container, H2 có thể vẫn còn) cho '{section_title_text}'.")

    if not content_parts:
        print(f"LỖI: Không thể trích xuất nội dung cụ thể nào cho mục '{section_title_text}'.")
        return "N/A"

    full_content = "\n".join(content_parts)
    return full_content.strip()


In [ ]:
async def get_job_description_from_page(page: Page) -> str:
    """
    Lấy Mô Tả Công Việc từ đối tượng page đã được điều hướng đến URL công việc.
    Sử dụng hàm trích xuất chung extract_generic_section.
    """
    return await extract_generic_section(page, "Mô Tả Công Việc")

async def get_job_requirements_from_page(page: Page) -> str:
    """
    Lấy Yêu Cầu Công Việc từ đối tượng page đã được điều hướng đến URL công việc.
    Sử dụng hàm trích xuất chung extract_generic_section.
    """
    return await extract_generic_section(page, "Yêu Cầu Công Việc")


In [ ]:
async def fetch_description_and_requirements_for_url(page: Page, job_url: str) -> dict:
    """
    Truy cập một URL công việc và trích xuất cả Mô tả và Yêu cầu công việc.
    Hàm này nhận một đối tượng page đã được tạo.
    """
    details = {
        "MoTaCongViec": "N/A",
        "YeuCauCongViec": "N/A"
    }
    try:
        print(f"Đang truy cập URL để lấy chi tiết: {job_url}")
        await page.goto(job_url, wait_until="domcontentloaded", timeout=60000)

        # Lấy Mô tả công việc
        print(f"--- Đang lấy Mô Tả Công Việc cho: {job_url} ---")
        details["MoTaCongViec"] = await get_job_description_from_page(page)

        # Lấy Yêu cầu công việc
        # Không cần page.goto() lại vì vẫn đang ở cùng trang
        print(f"--- Đang lấy Yêu Cầu Công Việc cho: {job_url} ---")
        details["YeuCauCongViec"] = await get_job_requirements_from_page(page)

    except Exception as e:
        print(f"LỖI NGHIÊM TRỌNG khi xử lý URL {job_url}: {e}")

    return details


# TEST THỬ CRAWL CÓ ĐƯỢC MÔ TẢ VỚI YÊU CẦU
async def example_run_for_single_url():
    """Ví dụ chạy hàm fetch_description_and_requirements_for_url cho một URL."""

    # --- THAY THẾ URL NÀY BẰNG URL CÔNG VIỆC BẠN MUỐN KIỂM TRA ---
    test_url = "https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-van-hanh-may-thu-nhap-tu-12-20-tr-thang.35C49CA2.html"
    # CỨ CHỌN LINK MUỐN TEST THỬ NHA, MIỄN LÀ NÓ CÒN TỒN TẠI TRONG TRANG CAREERVN

    if "URL_CONG_VIEC_THUC_TE_CUA_BAN" in test_url or not test_url.startswith("http"): # Kiểm tra placeholder
         print("Vui lòng thay thế 'test_url' bằng một URL công việc thực tế từ CareerViet để kiểm tra.")
         return

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
        )
        page = await context.new_page()

        job_details = await fetch_description_and_requirements_for_url(page, test_url)

        print("\n\n--- KẾT QUẢ TRÍCH XUẤT CHO URL TEST ---")
        print(f"URL: {test_url}")
        print("---------------------------------------")
        print("Mô Tả Công Việc:")
        print(job_details["MoTaCongViec"])
        print("---------------------------------------")
        print("Yêu Cầu Công Việc:")
        print(job_details["YeuCauCongViec"])
        print("---------------------------------------")

        await browser.close()


await example_run_for_single_url()


Đang truy cập URL để lấy chi tiết: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-van-hanh-may-thu-nhap-tu-12-20-tr-thang.35C49CA2.html
--- Đang lấy Mô Tả Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-van-hanh-may-thu-nhap-tu-12-20-tr-thang.35C49CA2.html ---
INFO: Đang tìm và trích xuất mục: 'Mô Tả Công Việc'
INFO: Không tìm thấy nội dung trong thẻ <p> cho 'Mô Tả Công Việc'. Thử tìm thẻ <li>.
INFO: Đã lấy nội dung cho 'Mô Tả Công Việc' từ thẻ <li>.
--- Đang lấy Yêu Cầu Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-van-hanh-may-thu-nhap-tu-12-20-tr-thang.35C49CA2.html ---
INFO: Đang tìm và trích xuất mục: 'Yêu Cầu Công Việc'
INFO: Không tìm thấy nội dung trong thẻ <p> cho 'Yêu Cầu Công Việc'. Thử tìm thẻ <li>.
INFO: Đã lấy nội dung cho 'Yêu Cầu Công Việc' từ thẻ <li>.


--- KẾT QUẢ TRÍCH XUẤT CHO URL TEST ---
URL: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-van-hanh-may-thu-nhap-tu-12-20-tr-thang.35C49CA2.html
----------------------

In [ ]:
async def process_dataframe_for_details(df_input):
    """
    Xử lý DataFrame đầu vào, lấy Mô tả và Yêu cầu cho mỗi URL.
    Thêm kết quả vào các cột mới: 'MoTaCongViec' và 'YeuCauCongViec'.
    """
    if 'URL' not in df_input.columns:
        print("LỖI: DataFrame đầu vào phải có cột 'URL'.")
        return df_input

    # Tạo danh sách để lưu kết quả
    descriptions_list = []
    requirements_list = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"
        )
        page = await context.new_page()

        total_urls = len(df_input)
        for index, row in df_input.iterrows():
            job_url = row['URL']
            print(f"\nĐang xử lý URL {index + 1}/{total_urls}: {job_url}")

            if pd.isna(job_url) or not isinstance(job_url, str) or not job_url.startswith("http"):
                print(f"CẢNH BÁO: URL không hợp lệ hoặc bị thiếu, bỏ qua: {job_url}")
                descriptions_list.append("N/A (URL không hợp lệ)")
                requirements_list.append("N/A (URL không hợp lệ)")
                continue

            # Gọi hàm đã tạo ở Ô 4 để lấy cả mô tả và yêu cầu
            details = await fetch_description_and_requirements_for_url(page, job_url)

            descriptions_list.append(details["MoTaCongViec"])
            requirements_list.append(details["YeuCauCongViec"])

            # Thêm một khoảng nghỉ nhỏ giữa các request để tránh làm quá tải server
            await asyncio.sleep(1) # Nghỉ 1 giây

        await browser.close()

    # Thêm các cột mới vào DataFrame
    df_output = df_input.copy() # Tạo bản sao để tránh thay đổi DataFrame gốc nếu không muốn
    df_output['MoTaCongViec'] = descriptions_list
    df_output['YeuCauCongViec'] = requirements_list

    return df_output

'''
scraped_data_df_test = pd.read_excel('/content/careerviet_jobs.xlsx')


from IPython.display import display # Để hiển thị đẹp trong Colab/Jupyter
display(scraped_data_df_test)

print("\nBắt đầu quá trình lấy Mô Tả và Yêu Cầu cho DataFrame...")
df_with_details = await process_dataframe_for_details(scraped_data_df_test.head(2)) # .head(2) để test nhanh với 2 dòng đầu

print("\nDataFrame sau khi thêm chi tiết:")
display(df_with_details)
'''


,TieuDeCongViec,TenCongTy,DiaDiem,MucLuong,HanNop,URL
0,TRƯỞNG PHÒNG MARKETING (MỚI),CÔNG TY TNHH BBQ VIỆT NAM,Hà Nội,Lương: 30 Tr - 35 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/truong-p...
1,CVCC Xử Lý Tín Dụng Khách Hàng Doanh Nghiệp Lớ...,Ngân Hàng TMCP Việt Nam Thịnh Vượng - VPBANK,Hồ Chí Minh,Lương: Cạnh tranh,10-07-2025,https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-...
2,Nhân Viên Vận Hành Kho (MỚI),CÔNG TY CỔ PHẦN KẾT CẤU THÉP REMARK,Đồng Nai,Lương: Cạnh tranh,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...
3,Cộng tác viên Kinh doanh (MỚI),CÔNG TY CỔ PHẦN DI CHUYỂN VÀ THÔNG MINH GSM,Hồ Chí Minh,Lương: 8 Tr - 12 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cong-tac...
4,Operation Manager (MỚI),CÔNG TY TNHH DỊCH VỤ THƯƠNG MẠI CUNG ỨNG AG,Hồ Chí Minh,Lương: 13 Tr - 15 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/operatio...
5,Cộng tác viên Hỗ trợ kinh doanh (MỚI),CÔNG TY CỔ PHẦN DI CHUYỂN VÀ THÔNG MINH GSM,Hồ Chí Minh,Lương: 6 Tr - 10 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cong-tac...
6,[HCM - DIST 7] - Finance Analyst (Finance Busi...,Talentnet Corporation,Hồ Chí Minh,Lương: 20 Tr - 25 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/hcm-dist...
7,Kỹ thuật viên trưởng xét nghiệm (MỚI),Công ty Cổ phần Tập đoàn Y Dược Vietlife,Hồ Chí Minh,Lương: 15 Tr - 25 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/ky-thuat...
8,Cloud Engineer (MỚI),CÔNG TY CỔ PHẦN VIỄN THÔNG DCNET,Hà Nội,Lương: Cạnh tranh,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cloud-en...
9,Nhân Viên Thiết Kế Đồ Họa Chuyển Động (Motion ...,CÔNG TY TNHH QUẢNG CÁO HYPER MEDIA,Hồ Chí Minh,Lương: 15 Tr - 25 Tr VND,07-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...



Bắt đầu quá trình lấy Mô Tả và Yêu Cầu cho DataFrame...

Đang xử lý URL 1/2: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html
Đang truy cập URL để lấy chi tiết: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html
--- Đang lấy Mô Tả Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html ---
INFO: Đang tìm và trích xuất mục: 'Mô Tả Công Việc'
INFO: Đã lấy nội dung cho 'Mô Tả Công Việc' từ thẻ <p>.
--- Đang lấy Yêu Cầu Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html ---
INFO: Đang tìm và trích xuất mục: 'Yêu Cầu Công Việc'
INFO: Đã lấy nội dung cho 'Yêu Cầu Công Việc' từ thẻ <p>.

Đang xử lý URL 2/2: https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-ly-tin-dung-khach-hang-doanh-nghiep-lon-hcm-ta163.35C49E1A.html
Đang truy cập URL để lấy chi tiết: https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-ly-tin-dung-khach-hang-doanh-nghiep-lon-hcm-ta163.35C49E1A.html
--- Đang lấy 

,TieuDeCongViec,TenCongTy,DiaDiem,MucLuong,HanNop,URL,MoTaCongViec,YeuCauCongViec
0,TRƯỞNG PHÒNG MARKETING (MỚI),CÔNG TY TNHH BBQ VIỆT NAM,Hà Nội,Lương: 30 Tr - 35 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/truong-p...,"Lập kế hoạch, thực thi và giám sát các chiến d...","Tốt nghiệp Đại học (Marketing, Quản trị kinh d..."
1,CVCC Xử Lý Tín Dụng Khách Hàng Doanh Nghiệp Lớ...,Ngân Hàng TMCP Việt Nam Thịnh Vượng - VPBANK,Hồ Chí Minh,Lương: Cạnh tranh,10-07-2025,https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-...,"Tiếp nhận, kiểm tra, soạn thảo và lưu các Hồ s...",QUYỀN LỢI:


In [ ]:
if 'scraped_data_df' in locals() and not scraped_data_df.empty:
 print("\nBắt đầu quá trình lấy Mô Tả và Yêu Cầu cho DataFrame chính của bạn...")
 scraped_data_df_updated = await process_dataframe_for_details(scraped_data_df)

 print("\nDataFrame chính sau khi thêm chi tiết:")
 display(scraped_data_df_updated)

else:
  print("Không tìm thấy DataFrame `scraped_data_df` hoặc DataFrame rỗng. Hãy đảm bảo nó đã được tạo.")



Bắt đầu quá trình lấy Mô Tả và Yêu Cầu cho DataFrame chính của bạn...

Đang xử lý URL 1/50: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html
Đang truy cập URL để lấy chi tiết: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html
--- Đang lấy Mô Tả Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html ---
INFO: Đang tìm và trích xuất mục: 'Mô Tả Công Việc'
INFO: Đã lấy nội dung cho 'Mô Tả Công Việc' từ thẻ <p>.
--- Đang lấy Yêu Cầu Công Việc cho: https://careerviet.vn/vi/tim-viec-lam/truong-phong-marketing.35C49E1B.html ---
INFO: Đang tìm và trích xuất mục: 'Yêu Cầu Công Việc'
INFO: Đã lấy nội dung cho 'Yêu Cầu Công Việc' từ thẻ <p>.

Đang xử lý URL 2/50: https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-ly-tin-dung-khach-hang-doanh-nghiep-lon-hcm-ta163.35C49E1A.html
Đang truy cập URL để lấy chi tiết: https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-ly-tin-dung-khach-hang-doanh-nghiep-lon-hcm-ta163.35C49E1A.ht

,TieuDeCongViec,TenCongTy,DiaDiem,MucLuong,HanNop,URL,MoTaCongViec,YeuCauCongViec
0,TRƯỞNG PHÒNG MARKETING (MỚI),CÔNG TY TNHH BBQ VIỆT NAM,Hà Nội,Lương: 30 Tr - 35 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/truong-p...,"Lập kế hoạch, thực thi và giám sát các chiến d...","Tốt nghiệp Đại học (Marketing, Quản trị kinh d..."
1,CVCC Xử Lý Tín Dụng Khách Hàng Doanh Nghiệp Lớ...,Ngân Hàng TMCP Việt Nam Thịnh Vượng - VPBANK,Hồ Chí Minh,Lương: Cạnh tranh,10-07-2025,https://careerviet.vn/vi/tim-viec-lam/cvcc-xu-...,"Tiếp nhận, kiểm tra, soạn thảo và lưu các Hồ s...",QUYỀN LỢI:
2,Nhân Viên Vận Hành Kho (MỚI),CÔNG TY CỔ PHẦN KẾT CẤU THÉP REMARK,Đồng Nai,Lương: Cạnh tranh,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,• Kiểm tra các chứng từ yêu cầu nhập/xuất hàng...,YÊU CẦU:\n• Trình độ chuyên môn: Tốt nghiệp Ca...
3,Cộng tác viên Kinh doanh (MỚI),CÔNG TY CỔ PHẦN DI CHUYỂN VÀ THÔNG MINH GSM,Hồ Chí Minh,Lương: 8 Tr - 12 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cong-tac...,Tiếp nhận và xử lý thông tin đăng ký từ mercha...,Sử dụng thành thạo laptop và email\nCó kỹ năng...
4,Operation Manager (MỚI),CÔNG TY TNHH DỊCH VỤ THƯƠNG MẠI CUNG ỨNG AG,Hồ Chí Minh,Lương: 13 Tr - 15 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/operatio...,- Kiểm tra chứng từ và khai tờ khai trên phần ...,YÊU CẦU:\n- Tốt nghiệp Cao đẳng trở lên chuyên...
5,Cộng tác viên Hỗ trợ kinh doanh (MỚI),CÔNG TY CỔ PHẦN DI CHUYỂN VÀ THÔNG MINH GSM,Hồ Chí Minh,Lương: 6 Tr - 10 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cong-tac...,"- Thu thập và xử lý thông tin, giấy tờ từ merc...",Sử dụng thành thạo laptop và email\nCó kỹ năng...
6,[HCM - DIST 7] - Finance Analyst (Finance Busi...,Talentnet Corporation,Hồ Chí Minh,Lương: 20 Tr - 25 Tr VND,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/hcm-dist...,N/A,N/A
7,Kỹ thuật viên trưởng xét nghiệm (MỚI),Công ty Cổ phần Tập đoàn Y Dược Vietlife,Hồ Chí Minh,Lương: 15 Tr - 25 Tr VND,30-06-2025,https://careerviet.vn/vi/tim-viec-lam/ky-thuat...,Lên kế hoạch phân công công việc cho nhân sự t...,1. Có chứng chỉ hành nghề\n2. Có tối thiểu 5 n...
8,Cloud Engineer (MỚI),CÔNG TY CỔ PHẦN VIỄN THÔNG DCNET,Hà Nội,Lương: Cạnh tranh,06-07-2025,https://careerviet.vn/vi/tim-viec-lam/cloud-en...,A. Cloud Infrastructure: Xây dụng và vận hành ...,• Có kinh nghiệm triển khai Openstack và hạ tầ...
9,Nhân Viên Thiết Kế Đồ Họa Chuyển Động (Motion ...,CÔNG TY TNHH QUẢNG CÁO HYPER MEDIA,Hồ Chí Minh,Lương: 15 Tr - 25 Tr VND,07-07-2025,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Chịu trách nhiệm thiết kế và sản xuất video qu...,"Tốt nghiệp cao đẳng trở lên, chuyên ngành Sản ..."
